# 🤖 LangGraph 多 Agent 系统实战

本 Notebook 展示如何使用 LangGraph 构建复杂的多 Agent 协作系统。

## 📋 学习目标

- LangGraph 基础概念（State、Node、Edge）
- 构建状态机工作流
- 多 Agent 协作模式
- 条件路由和决策
- 使用 LangSmith 监控调试
- 实战案例：智能内容创作团队


## 📦 安装依赖


In [ ]:
%pip install -q langgraph langchain langchain-openai langsmith


## 🔧 配置 DeepSeek API 和 LangSmith


In [ ]:
import os
from typing import TypedDict, Annotated, Sequence
from langchain_openai import ChatOpenAI

print("🔧 配置 DeepSeek API 和 LangSmith")
print("="*60)

# DeepSeek API 配置检查
api_key = os.environ.get("OPENAI_API_KEY", "")
api_base = os.environ.get("OPENAI_API_BASE", "https://api.deepseek.com")

if not api_key:
    print("\n⚠️  警告: 未检测到 DeepSeek API Key！")
    print("请在终端运行：")
    print("   export OPENAI_API_KEY='your-deepseek-api-key'")
    print("   export OPENAI_API_BASE='https://api.deepseek.com'")
else:
    print(f"\n✅ DeepSeek API 已配置")
    print(f"✅ API Base: {api_base}")

# LangSmith 配置检查
langsmith_key = os.environ.get("LANGCHAIN_API_KEY", "")
langsmith_tracing = os.environ.get("LANGCHAIN_TRACING_V2", "")
langsmith_project = os.environ.get("LANGCHAIN_PROJECT", "langgraph-multi-agent")

if not langsmith_key or langsmith_tracing != "true":
    print("\n⚠️  警告: LangSmith 未完全配置！")
    print("请在终端运行：")
    print("   export LANGCHAIN_TRACING_V2='true'")
    print("   export LANGCHAIN_API_KEY='your-langsmith-api-key'")
    print("   export LANGCHAIN_PROJECT='langgraph-multi-agent'")
    print("\n访问 https://smith.langchain.com/ 获取 API Key")
else:
    print(f"\n✅ LangSmith 监控已启用")
    print(f"✅ Project: {langsmith_project}")
    print(f"✅ 访问 https://smith.langchain.com/ 查看执行详情")

print("="*60)

# 初始化 LLM
llm = ChatOpenAI(
    model="deepseek-chat",
    openai_api_base="https://api.deepseek.com",
    temperature=0.7
)

print("\n✅ 配置完成！所有执行将自动上传到 LangSmith 平台")


## 一、LangGraph 基础概念

### 核心概念

```
State (状态)
   ↓
Node (节点/Agent)
   ↓
Edge (边/转换)
   ↓
Graph (整个工作流)
```

### 简单示例


In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict

# 1. 定义状态
class SimpleState(TypedDict):
    messages: list[str]
    counter: int

# 2. 定义节点函数
def step1(state: SimpleState):
    return {
        "messages": state["messages"] + ["步骤1完成"],
        "counter": state["counter"] + 1
    }

def step2(state: SimpleState):
    return {
        "messages": state["messages"] + ["步骤2完成"],
        "counter": state["counter"] + 1
    }

# 3. 构建图
workflow = StateGraph(SimpleState)
workflow.add_node("step1", step1)
workflow.add_node("step2", step2)

# 4. 添加边
workflow.set_entry_point("step1")
workflow.add_edge("step1", "step2")
workflow.add_edge("step2", END)

# 5. 编译并运行
app = workflow.compile()

result = app.invoke({"messages": [], "counter": 0})
print("\n结果:")
print(f"消息: {result['messages']}")
print(f"计数: {result['counter']}")
print("\n💡 所有执行细节已自动上传到 LangSmith，请访问控制台查看可视化流程！")


## 二、多 Agent 内容创作系统

### 2.1 定义状态


In [ ]:
from typing import TypedDict, List
import operator
from langchain.schema import HumanMessage, AIMessage

class AgentState(TypedDict):
    topic: str
    research_notes: str
    draft_content: str
    final_content: str
    messages: Annotated[List, operator.add]
    next_agent: str
    iteration: int
    max_iterations: int


### 2.2 创建各个 Agent


In [ ]:
from langchain.prompts import ChatPromptTemplate

# 研究员 Agent
def researcher_agent(state: AgentState) -> AgentState:
    prompt = ChatPromptTemplate.from_template(
        """你是一个专业的研究员。请针对以下主题进行研究，提供关键要点和有价值的信息。

主题：{topic}

请提供：
1. 核心概念定义
2. 重要特点（3-5个）
3. 实际应用场景
4. 注意事项

研究笔记："""
    )
    
    chain = prompt | llm
    result = chain.invoke({"topic": state["topic"]})
    
    return {
        **state,
        "research_notes": result.content,
        "messages": [f"[研究员] 完成研究: {state['topic'][:50]}..."],
        "next_agent": "writer"
    }

# 作家 Agent
def writer_agent(state: AgentState) -> AgentState:
    prompt = ChatPromptTemplate.from_template(
        """你是一个专业的技术作家。基于研究员提供的笔记，撰写一篇结构清晰、通俗易懂的文章。

主题：{topic}

研究笔记：
{research_notes}

要求：
- 使用清晰的标题和段落结构
- 语言通俗易懂，适合初学者
- 包含实例说明
- 字数约500字

文章内容："""
    )
    
    chain = prompt | llm
    result = chain.invoke({
        "topic": state["topic"],
        "research_notes": state["research_notes"]
    })
    
    return {
        **state,
        "draft_content": result.content,
        "messages": [f"[作家] 完成初稿撰写"],
        "next_agent": "editor"
    }

# 编辑 Agent
def editor_agent(state: AgentState) -> AgentState:
    prompt = ChatPromptTemplate.from_template(
        """你是一个专业的编辑。请审核并改进以下文章。

原文：
{draft_content}

请：
1. 检查语法和表达
2. 优化结构和逻辑
3. 确保内容准确性
4. 增强可读性

如果文章质量已经很好，保持原样并添加简短评价。
如果需要改进，请直接输出改进后的版本。

最终文章："""
    )
    
    chain = prompt | llm
    result = chain.invoke({"draft_content": state["draft_content"]})
    
    new_iteration = state["iteration"] + 1
    
    return {
        **state,
        "final_content": result.content,
        "messages": [f"[编辑] 完成审核和改进"],
        "next_agent": "END",
        "iteration": new_iteration
    }

print("✅ Agent 定义完成！")


In [ ]:
from langgraph.graph import StateGraph, END

# 路由函数：决定下一个节点
def route_agent(state: AgentState) -> str:
    next_agent = state.get("next_agent", "researcher")
    if next_agent == "END":
        return END
    return next_agent

# 创建图
workflow = StateGraph(AgentState)

# 添加节点
workflow.add_node("researcher", researcher_agent)
workflow.add_node("writer", writer_agent)
workflow.add_node("editor", editor_agent)

# 设置入口
workflow.set_entry_point("researcher")

# 添加条件边
workflow.add_conditional_edges(
    "researcher",
    route_agent,
    {"writer": "writer", END: END}
)

workflow.add_conditional_edges(
    "writer",
    route_agent,
    {"editor": "editor", END: END}
)

workflow.add_conditional_edges(
    "editor",
    route_agent,
    {"writer": "writer", END: END}
)

# 编译
app = workflow.compile()

print("✅ 工作流构建完成！")


### 2.4 运行多 Agent 系统

所有执行细节会自动上传到 LangSmith 平台，你可以：
- 查看每个 Agent 的执行时间
- 查看每次 LLM 调用的 Prompt 和响应
- 分析 Token 使用和成本
- 可视化整个工作流的执行路径


In [ ]:
# 初始化状态
initial_state = {
    "topic": "深度学习中的 Transformer 架构",
    "research_notes": "",
    "draft_content": "",
    "final_content": "",
    "messages": [],
    "next_agent": "researcher",
    "iteration": 0,
    "max_iterations": 2
}

print("🚀 启动多 Agent 内容创作系统...")
print(f"📝 主题: {initial_state['topic']}\n")
print("="*60)
print("\n💡 执行过程正在上传到 LangSmith...")
print("   访问 https://smith.langchain.com/ 查看实时执行详情\n")
print("="*60 + "\n")

# 运行工作流
final_state = app.invoke(initial_state)

print("\n" + "="*60)
print("\n🎉 创作完成！")
print("\n📋 工作流程:")
for msg in final_state["messages"]:
    print(f"  {msg}")

print("\n" + "="*60)
print("\n📄 最终文章:")
print(final_state["final_content"][:500] + "...\n")

print("="*60)
print("\n🔍 想查看详细执行过程？")
print("   访问 LangSmith 平台查看：")
print("   - 每个 Agent 的执行时间")
print("   - LLM 调用的完整 Prompt 和响应")
print("   - Token 使用统计和成本")
print("   - 工作流可视化图表")
print("\n   https://smith.langchain.com/")
print("="*60)


## 三、高级特性：动态路由和循环

### 3.1 带质量检查的循环改进


In [ ]:
# 质量评估 Agent
def quality_checker(state: AgentState) -> AgentState:
    prompt = ChatPromptTemplate.from_template(
        """评估以下文章的质量，并给出1-10的分数。

文章：
{content}

请只回答一个数字（1-10）和简短评价（一句话）。
格式：分数: X, 评价: XXX
"""
    )
    
    chain = prompt | llm
    result = chain.invoke({"content": state["final_content"]})
    
    try:
        score_str = result.content.split("分数:")[1].split(",")[0].strip()
        score = int(score_str)
    except:
        score = 8
    
    new_iteration = state["iteration"] + 1
    if score < 8 and new_iteration < state["max_iterations"]:
        next_agent = "writer"
        message = f"[质检] 分数{score}/10, 需要改进"
    else:
        next_agent = "END"
        message = f"[质检] 分数{score}/10, 质量达标"
    
    return {
        **state,
        "messages": [message],
        "next_agent": next_agent,
        "iteration": new_iteration
    }

print("✅ 质量检查 Agent 定义完成！")
print("\n💡 在 LangSmith 中可以看到质量检查的循环迭代过程")


## 四、总结

### ✅ 掌握的技能

1. **LangGraph 基础**
   - State 状态管理
   - Node 节点定义
   - Edge 边和路由
   - Graph 工作流编排

2. **多 Agent 协作**
   - Agent 角色分工
   - 状态传递和共享
   - 条件路由
   - 循环和迭代

3. **LangSmith 监控**
   - 自动追踪所有执行
   - 可视化工作流
   - 性能和成本分析
   - 调试和优化

### 🚀 应用场景

- 📝 内容创作团队
- 🎯 客户服务系统
- 🔍 代码审查流程
- 📊 数据分析管道
- 🤖 智能助手系统

### 💡 最佳实践

1. **清晰的状态设计**：定义完整的状态结构
2. **单一职责**：每个 Agent 专注一个任务
3. **使用 LangSmith**：监控所有执行细节
4. **错误处理**：考虑异常情况和回退机制
5. **性能优化**：通过 LangSmith 分析瓶颈

---

**🎉 恭喜！你已经掌握了 LangGraph 多 Agent 系统和 LangSmith 监控的核心技能！**

**🔍 下一步：**
访问 [https://smith.langchain.com/](https://smith.langchain.com/) 查看你的执行记录，分析性能和成本！
